# Chapter6-3 文書分類


## 6-3-2 前処理

In [ ]:
from glob import glob

In [ ]:
directories = glob('text/*')
directories

In [ ]:
txts = glob('text/*.txt')
txts

In [ ]:
for txt in txts:
    directories.remove(txt)

In [ ]:
directories

In [ ]:
filepaths = glob('text/it-life-hack/*.txt')

In [ ]:
filepath = filepaths[0]
filepath

In [ ]:
with open(filepath, encoding='utf-8') as f:
    # URL等の先頭２行を除いた各行の文章を連結（join）して格納
    text = ''.join(f.readlines()[2:])

In [ ]:
# 先頭の 150 文字を表示
text[:150]

In [ ]:
from janome.tokenizer import Tokenizer

In [ ]:
tagger = Tokenizer(wakati=True)

In [ ]:
words = tagger.tokenize(text)
words[:10]

In [ ]:
def preprocessing(filepath):
    with open(filepath, encoding='utf-8') as f:
        # URL等の先頭２行を除いた各行の文章を連結（join）して格納
        text = ''.join(f.readlines()[2:])
        text = text.replace('\u3000', '')
        text = text.replace('\n', '')
        words = tagger.tokenize(text)
    return words

In [ ]:
words = preprocessing(filepath)
words[:10]

In [ ]:
def labeling(directory):
    if 'it-life-hack' in directory or 'kaden-channel' in directory:
        return 1
    else:
        return 0

In [ ]:
labeling('text/it-life-hack')

In [ ]:
labeling('text/movie-enter')

In [ ]:
word_collect, labels = [], []
for directory in directories:
    filepaths = glob(directory + '/*.txt')
    for filepath in filepaths:
        words = preprocessing(filepath)
        label = labeling(directory)
        word_collect.append(words)
        labels.append(label)

In [ ]:
len(word_collect)

In [ ]:
from gensim import corpora, matutils

In [ ]:
dictionary = corpora.Dictionary(word_collect)

In [ ]:
n_words = len(dictionary)
n_words

In [ ]:
dictionary.filter_extremes(no_below=20)

In [ ]:
n_words = len(dictionary)
n_words

In [ ]:
bow_ids = []
for words in word_collect:
    bow_id = dictionary.doc2bow(words)
    bow_ids.append(bow_id)

In [ ]:
bows = matutils.corpus2dense(bow_ids, n_words).T

In [ ]:
bows.shape

In [ ]:
import numpy as np

In [ ]:
# i は データ型を int32 と指定
labels = np.array(labels, 'i')

## 6-3-3 モデル構築

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
plt.hist(labels, bins=3)

In [ ]:
weight = {
    0: len(labels) / len(labels[labels==0]),
    1: len(labels) / len(labels[labels==1])
}

In [ ]:
weight

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, t_train, t_test = train_test_split(bows, labels, test_size=0.3, random_state=0)

In [ ]:
x_train.shape, x_test.shape, t_train.shape, t_test.shape

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
clf = DecisionTreeClassifier(class_weight=weight, random_state=0)

In [ ]:
clf.fit(x_train, t_train)

In [ ]:
clf.score(x_train, t_train)

In [ ]:
clf.score(x_test, t_test)

In [ ]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix

In [ ]:
y_test = clf.predict(x_test)

In [ ]:
# 適合率
precision_score(y_test, t_test)

In [ ]:
# 再現率
recall_score(y_test, t_test)

In [ ]:
# 混合行列
matrix = confusion_matrix(t_test, y_test)
matrix

In [ ]:
import seaborn as sns

In [ ]:
# 混合行列の可視化
sns.heatmap(matrix, annot=True, cmap='Blues');
plt.xlabel('Prediction')
plt.ylabel('Target')

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# ハイパーパラメータの候補を列挙
params = {
    "max_depth": list(range(2, 5)),
    "criterion": ["gini", "entropy"]
}

In [ ]:
# 評価する scoring に precision を指定
# cv は交差検証の分割数
clf_grid = GridSearchCV(
    estimator=DecisionTreeClassifier(class_weight=weight, random_state=0),
    scoring='precision',
    param_grid=params,
    cv=5,
)

In [ ]:
# すべてのハイパーパラメータの候補で訓練
clf_grid.fit(x_train, t_train)

In [ ]:
# 最も良かったハイパーパラメータの組み合わせ
clf_grid.best_params_

In [ ]:
# 最も良かったハイパーパラメータを持つ訓練済みモデルを受け継ぐ
clf = clf_grid.best_estimator_
clf

In [ ]:
# 正解率
clf.score(x_test, t_test)

In [ ]:
# 予測値の計算
y_test = clf.predict(x_test)

In [ ]:
# 適合率
precision_score(t_test, y_test)

In [ ]:
# 再現率
recall_score(t_test, y_test)

In [ ]:
# 混合行列
matrix = confusion_matrix(t_test, y_test)
matrix

In [ ]:
# 混合行列の可視化
sns.heatmap(matrix, annot=True, cmap='Blues');
plt.xlabel('Prediction')
plt.ylabel('Target')

In [ ]:
import joblib

In [ ]:
# モデルの保存
joblib.dump(clf, 'document_classifier.pkl')

In [ ]:
# 辞書の保存
joblib.dump(dictionary, 'dictionary.pkl')